 # Night glucose, nutrition and lifestyle
    
 Looking for associations between lifestyle data, nutrition and night glucose levels

In [1]:
import pandas as pd

In [2]:
import LabData
from LabData.DataLoaders.Loader import Loader

In [3]:
cgmLoader = Loader.get_loader("CGMLoader")

In [95]:
cgm = cgmLoader.get_data(study_ids=3)
#cgm = cgmLoader.get_data()

In [96]:
cgm_df = cgm.df
cgm_meta = cgm.df_metadata

In [97]:
cgm_df

GlucoseValue     PPGR
RegistrationCode ConnectionID Date                                            
111527           1926         2017-11-06 14:16:00+02:00          79.0  109.750
                              2017-11-06 14:31:00+02:00          83.0  132.500
                              2017-11-06 14:46:00+02:00          86.0  153.500
                              2017-11-06 15:01:00+02:00          89.0  170.750
                              2017-11-06 15:16:00+02:00         118.0  127.375
...                                                               ...      ...
997735           5832         2019-07-21 07:45:00+03:00         117.0      NaN
                              2019-07-21 08:00:00+03:00         119.0      NaN
                              2019-07-21 08:15:00+03:00         127.0      NaN
                              2019-07-21 08:30:00+03:00         138.0      NaN
                              2019-07-21 08:45:00+03:00         135.0      NaN

[3478385 rows x 2 columns]

In [7]:
cgm_df.loc[('997735', slice(None), slice(None)), :]

GlucoseValue  PPGR
RegistrationCode ConnectionID Date                                         
997735           4596         2018-12-31 13:12:00+02:00         107.0   0.0
                              2018-12-31 13:27:00+02:00         108.0   0.0
                              2018-12-31 13:42:00+02:00         106.0   0.0
                              2018-12-31 13:57:00+02:00         102.0   0.0
                              2018-12-31 14:12:00+02:00          96.0   0.0
...                                                               ...   ...
                 5832         2019-07-21 07:45:00+03:00         117.0   NaN
                              2019-07-21 08:00:00+03:00         119.0   NaN
                              2019-07-21 08:15:00+03:00         127.0   NaN
                              2019-07-21 08:30:00+03:00         138.0   NaN
                              2019-07-21 08:45:00+03:00         135.0   NaN

[15241 rows x 2 columns]

In [8]:
def filter_by_time(df, start, end):
    
    """
    The function filters the cgm entries between certain hours of the day. 
    The output is a dataframe containing the entries between start and end hour of each day.
    :param df: CGM DataFrame
    :param start: string of a type 'hh:mm'
    :param end: string of a type 'hh:mm'
    :return: filtered DataFrame with an index set to 'RegistrationCode', 'Day' and 'Time'
    """

    df = df.reset_index(['RegistrationCode', 'ConnectionID'])
    filtered_df = df.between_time(start, end)
    pd.options.mode.chained_assignment = None  # to avoid the warning message about chained assignments
    filtered_df['Day'] = filtered_df.index.date
    filtered_df['Time'] = filtered_df.index - filtered_df.index.normalize()
    filtered_df = filtered_df.set_index(['RegistrationCode', 'Day', 'Time'])
    filtered_df = filtered_df.sort_index(level=['RegistrationCode', 'Day', 'Time'])
    
    return filtered_df

In [9]:
filter_by_time(cgm_df, '1:00', '10:00')

ConnectionID  GlucoseValue   PPGR
RegistrationCode Day        Time                                      
111527           2017-11-07 01:00:00         1926          97.0   0.00
                            01:15:00         1926          99.0   0.00
                            01:30:00         1926          94.0   0.00
                            01:45:00         1926          82.0  16.50
                            02:00:00         1926          89.0   2.25
...                                           ...           ...    ...
997735           2019-07-21 07:45:00         5832         117.0    NaN
                            08:00:00         5832         119.0    NaN
                            08:15:00         5832         127.0    NaN
                            08:30:00         5832         138.0    NaN
                            08:45:00         5832         135.0    NaN

[1331383 rows x 3 columns]

## Sleep data

In [118]:
sleep_df = pd.read_pickle('/net/mraid08/export/genie/LabData/Data/WIS/sleep_wakeup.dat')

/usr/wisdom/python3.5.3/lib/python3.5/site-packages/pandas/core/indexes/base.py:183: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)


In [121]:
sleep_df.reset_index('RegistrationCode', inplace=True)

In [122]:
sleep_df.loc[:, ['RegistrationCode']] = sleep_df.loc[:,'RegistrationCode'].astype(int).astype(str)

In [123]:
sleep_df

RegistrationCode Quality          sleep_time         wakeup_time
RunningIndex                                                                 
0                           0     NaT 2015-06-11 09:23:15 2015-06-11 09:23:21
1                           0     NaT 2015-06-11 09:56:54 2015-06-11 19:56:37
2                           0     NaT 2015-06-12 01:50:00 2015-06-12 02:29:19
3                           0     NaT 2015-06-12 02:29:09 2015-06-12 02:29:19
4                           0     NaT 2015-06-14 01:45:40 2015-06-14 11:08:19
...                       ...     ...                 ...                 ...
49756              2147483647     NaT 2015-05-22 13:32:28 2015-05-23 07:10:41
49757              2147483647     NaT 2015-05-23 01:30:34 2015-05-23 07:10:41
49758              2147483647     NaT 2015-05-24 00:24:00 2015-05-25 11:00:43
49759              2147483647     NaT 2015-05-25 00:19:00 2015-05-25 11:00:43
49760              2147483647     NaT 2015-05-27 01:40:48 2015-05-27 07:30:47

[49761 rows x 4 columns]

## CGM Metrics

In [124]:
cgm_df

RegistrationCode ConnectionID                      Date  GlucoseValue  \
0                 111527         1926 2017-11-06 14:16:00+02:00          79.0   
1                 111527         1926 2017-11-06 14:31:00+02:00          83.0   
2                 111527         1926 2017-11-06 14:46:00+02:00          86.0   
3                 111527         1926 2017-11-06 15:01:00+02:00          89.0   
4                 111527         1926 2017-11-06 15:16:00+02:00         118.0   
...                  ...          ...                       ...           ...   
3478380           997735         5832 2019-07-21 07:45:00+03:00         117.0   
3478381           997735         5832 2019-07-21 08:00:00+03:00         119.0   
3478382           997735         5832 2019-07-21 08:15:00+03:00         127.0   
3478383           997735         5832 2019-07-21 08:30:00+03:00         138.0   
3478384           997735         5832 2019-07-21 08:45:00+03:00         135.0   

            PPGR  
0        109.750  
1        132.500  
2        153.500  
3        170.750  
4        127.375  
...          ...  
3478380      NaN  
3478381      NaN  
3478382      NaN  
3478383      NaN  
3478384      NaN  

[3478385 rows x 5 columns]

In [126]:
cgm_df['day'] = cgm_df['Date'].dt.date
cgm_df

RegistrationCode ConnectionID                      Date  GlucoseValue  \
0                 111527         1926 2017-11-06 14:16:00+02:00          79.0   
1                 111527         1926 2017-11-06 14:31:00+02:00          83.0   
2                 111527         1926 2017-11-06 14:46:00+02:00          86.0   
3                 111527         1926 2017-11-06 15:01:00+02:00          89.0   
4                 111527         1926 2017-11-06 15:16:00+02:00         118.0   
...                  ...          ...                       ...           ...   
3478380           997735         5832 2019-07-21 07:45:00+03:00         117.0   
3478381           997735         5832 2019-07-21 08:00:00+03:00         119.0   
3478382           997735         5832 2019-07-21 08:15:00+03:00         127.0   
3478383           997735         5832 2019-07-21 08:30:00+03:00         138.0   
3478384           997735         5832 2019-07-21 08:45:00+03:00         135.0   

            PPGR         day  
0        109.750  2017-11-06  
1        132.500  2017-11-06  
2        153.500  2017-11-06  
3        170.750  2017-11-06  
4        127.375  2017-11-06  
...          ...         ...  
3478380      NaN  2019-07-21  
3478381      NaN  2019-07-21  
3478382      NaN  2019-07-21  
3478383      NaN  2019-07-21  
3478384      NaN  2019-07-21  

[3478385 rows x 6 columns]

In [133]:
means = cgm_df
means = means.drop(['ConnectionID', 'Date', 'PPGR'], axis=1)

In [134]:
means

RegistrationCode  GlucoseValue         day
0                 111527          79.0  2017-11-06
1                 111527          83.0  2017-11-06
2                 111527          86.0  2017-11-06
3                 111527          89.0  2017-11-06
4                 111527         118.0  2017-11-06
...                  ...           ...         ...
3478380           997735         117.0  2019-07-21
3478381           997735         119.0  2019-07-21
3478382           997735         127.0  2019-07-21
3478383           997735         138.0  2019-07-21
3478384           997735         135.0  2019-07-21

[3478385 rows x 3 columns]

In [135]:
test = means.groupby(['RegistrationCode', 'day'], as_index=False).agg(
                      {'GlucoseValue':['mean','std']})

In [136]:
test

RegistrationCode         day GlucoseValue           
                                           mean        std
0               111527  2017-11-06   113.897436  38.182246
1               111527  2017-11-07   110.072165  21.003595
2               111527  2017-11-08   113.020833  19.443294
3               111527  2017-11-09   110.621053  22.600216
4               111527  2017-11-10   109.989583  22.246227
...                ...         ...          ...        ...
43662           997735  2019-07-17   108.750000  14.211930
43663           997735  2019-07-18   112.376344  12.823140
43664           997735  2019-07-19   109.163043  10.772653
43665           997735  2019-07-20   114.458333  34.610286
43666           997735  2019-07-21   111.428571   9.249120

[43667 rows x 4 columns]